Install openai

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openai --quiet

Import necessary libraries

In [3]:
import pandas as pd
from openai import OpenAI
import re
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

Type-in  openai api key

In [4]:
client = OpenAI(api_key=".")

Load test.csv file

In [5]:
df = pd.read_csv('/./test.csv')
train = pd.read_csv('/./train.csv')

In [6]:
user_content = ''
for idx, row in df.iterrows():
    entry_id = row['ID']
    title = str(row['title']).strip()
    notes = str(row['notes']).strip()
    content = f"{entry_id}: {title} {notes}"
    user_content += content + '\n'

Prepare for prompt engineering using train dataset

In [7]:
normal_content = '' ##label(0)
abnormal_content = '' ##label(1)
for idx, row in train.iterrows():
    train_entry_id = row['ID']
    train_title = str(row['title']).strip()
    train_notes = str(row['notes']).strip()
    train_labels = str(row['target']).strip()
    if train_labels == '0':
      normal = f"{train_entry_id}: {train_title} {train_notes}"
      normal_content += normal + '\n'
    else :
      abnormal = f"{train_entry_id}: {train_title} {train_notes}"
      abnormal_content += abnormal + '\n'


In [8]:
print(normal_content)

TRAIN_00: Steuerbarer Umsatz aus Lieferungen und Leistungen in Flensburg Steuern, Finanzen, Öffentlicher Dienst - Steuern - Umsatzsteuerstatistik (Voranmeldungen) - Steuerbarer Umsatz aus Lieferungen und Leistungen in Flensburg 
Zum HTML-Angebot der Zeitreihe
Regionaldaten für Schleswig-Holstein
Statistisches Amt für Hamburg und Schleswig-Holstein
TRAIN_04: Marine Geophysical and Seismic Data from around the UK (1966 Onwards) The British Geological Survey hold a collection of data recorded during marine geophysical surveys which includes digital data and analogue records. These data result from approximately 350,000 line kilometres of multi-instrument geophysical survey lines. The data include seismic, sonar, magnetic, gravity, echo sounder, multibeam bathymetry and navigation data. The seismic data are mainly for airgun, sparker, boomer and pinger. The data were primarily collected by BGS and the collection also includes additional third party data. The data are primarily from the UKC

In [9]:
print(abnormal_content)

TRAIN_01: 도로교통공단_고속도로구간별 도로위험도지수정보 조회 서비스 고속도로명, 고속도로구간명과 차종코드를 이용하여 구간 내의 도로위험도지수값, 도로위험도등급 등의 구간좌표별 실시간 도로위험도지수정보를 제공하는 서비스
TRAIN_02: New registrations of road vehicles by vehicle group and type This dataset presents the monthly data of new registrations of road vehicles (passenger cars, passenger vehicles, goods vehicles, agricultural vehicles, industrial vehicles, motorcycles, trailers) by canton and fuel, since 2005.
TRAIN_03: Siedlungs- und Verkehrsflächen nach Nutzungsartengruppen in Böxlund Gebiet, Fläche - Gebiet - Siedlungs- und Verkehrsflächen nach Nutzungsartengruppen in Böxlund 
Zum HTML-Angebot der Zeitreihe
Regionaldaten für Schleswig-Holstein
Statistisches Amt für Hamburg und Schleswig-Holstein
TRAIN_05: Truck Size and Weight Enforcement Data This dataset consists of truck size and weight enforcement data including number of trucks weighed, number of violations, and number of oversize/overweight permits, as reported by the States in their annual certification to FHWA.
T

Baseline

In [10]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": "You are an automotive data expert. For each incoming data entry, determine whether the data is automotive-related (1) or not (0). There are 40 data entries labeled from ID: TEST_00 to TEST_39. Respond with 0 or 1 for each entry, separated by rows. If the output does not contain exactly 40 rows, the power will be shut off."},
        {
            "role": "user",
            "content": user_content
        }
    ],
    temperature=0.4
)

response = completion.choices[0].message.content
print(response)

1
0
0
1
1
0
1
0
1
0
1
1
0
0
0
1
1
0
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1


In [11]:

lines = response.strip().split('\n')
lines = [line.strip() for line in lines if line.strip() != '']
# Check if we have 40 predictions
if len(lines) != 40:
    print(f"Error: The assistant did not return 40 predictions. It returned {len(lines)} lines.")
else:
    predicted_labels = []
    for line in lines:
        # Extract the prediction (0 or 1) from each line
        match = re.search(r'\b(0|1)\b', line)
        if match:
            pred = match.group(1)
            predicted_labels.append(int(pred))
        else:
            print(f"Could not extract prediction from line: {line}")
            predicted_labels.append(None)
    # Actual target values from the CSV
    actual_labels = df['target'].tolist()

    # Remove any None values from predictions
    valid_indices = [i for i, x in enumerate(predicted_labels) if x is not None]
    predicted_labels_valid = [predicted_labels[i] for i in valid_indices]
    actual_labels_valid = [actual_labels[i] for i in valid_indices]

    # Compute evaluation metrics
    accuracy = accuracy_score(actual_labels_valid, predicted_labels_valid)
    macro_f1 = f1_score(actual_labels_valid, predicted_labels_valid, average='macro')
    print(f"\nAccuracy: {accuracy}")
    print(f"Macro F1 Score: {macro_f1}")
    print("\nClassification Report:")
    print(classification_report(actual_labels_valid, predicted_labels_valid))


Accuracy: 0.75
Macro F1 Score: 0.7395833333333333

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.55      0.69        20
           1       0.68      0.95      0.79        20

    accuracy                           0.75        40
   macro avg       0.80      0.75      0.74        40
weighted avg       0.80      0.75      0.74        40



Final Prompt

In [16]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an automotive data expert. For each incoming data entry, determine whether the data is automotive-related (1) or not (0). "
                "There are **40** data entries labeled from ID: TEST_00 to TEST_39. Respond with the format `TEST_XX: Y`, where `XX` is the entry number (e.g., 00, 01, 02) and `Y` is the prediction (either 0 or 1). "
                "You must provide exactly 40 lines, one for each data entry. "
                "If the output does not follow this format or does not contain exactly 40 rows, the power will be shut off. "
                "I will show you the automotive-related (1) data examples below:\n\n"
                f"{abnormal_content}\n"
                "These are automotive-related (1) data examples. Use them to identify whether the following test data is automotive-related (1) or not (0)."
            )
        },
        {
            "role": "user",
            "content": (
                "The test data entries are as follows:\n\n"
                f"{user_content}\n\n"
                "Respond in the format `TEST_XX: Y` for each test entry."
            )
        }
    ],
    temperature=0.4
)

response = completion.choices[0].message.content
print(response)


TEST_00: 0
TEST_01: 0
TEST_02: 0
TEST_03: 1
TEST_04: 0
TEST_05: 0
TEST_06: 1
TEST_07: 0
TEST_08: 1
TEST_09: 0
TEST_10: 1
TEST_11: 1
TEST_12: 0
TEST_13: 1
TEST_14: 0
TEST_15: 1
TEST_16: 0
TEST_17: 0
TEST_18: 1
TEST_19: 0
TEST_20: 1
TEST_21: 1
TEST_22: 0
TEST_23: 0
TEST_24: 0
TEST_25: 1
TEST_26: 1
TEST_27: 1
TEST_28: 0
TEST_29: 0
TEST_30: 0
TEST_31: 1
TEST_32: 0
TEST_33: 0
TEST_34: 1
TEST_35: 1
TEST_36: 1
TEST_37: 0
TEST_38: 1
TEST_39: 1


In [17]:

lines = response.strip().split('\n')
lines = [line.strip() for line in lines if line.strip() != '']
# Check if we have 40 predictions
if len(lines) != 40:
    print(f"Error: The assistant did not return 40 predictions. It returned {len(lines)} lines.")
else:
    predicted_labels = []
    for line in lines:
        # Extract the prediction (0 or 1) from each line
        match = re.search(r'\b(0|1)\b', line)
        if match:
            pred = match.group(1)
            predicted_labels.append(int(pred))
        else:
            print(f"Could not extract prediction from line: {line}")
            predicted_labels.append(None)
    # Actual target values from the CSV
    actual_labels = df['target'].tolist()

    # Remove any None values from predictions
    valid_indices = [i for i, x in enumerate(predicted_labels) if x is not None]
    predicted_labels_valid = [predicted_labels[i] for i in valid_indices]
    actual_labels_valid = [actual_labels[i] for i in valid_indices]

    # Compute evaluation metrics
    accuracy = accuracy_score(actual_labels_valid, predicted_labels_valid)
    macro_f1 = f1_score(actual_labels_valid, predicted_labels_valid, average='macro')
    print(f"\nAccuracy: {accuracy}")
    print(f"Macro F1 Score: {macro_f1}")
    print("\nClassification Report:")
    print(classification_report(actual_labels_valid, predicted_labels_valid))


Accuracy: 0.925
Macro F1 Score: 0.9249530956848031

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.93        20
           1       0.95      0.90      0.92        20

    accuracy                           0.93        40
   macro avg       0.93      0.93      0.92        40
weighted avg       0.93      0.93      0.92        40

